In [5]:

import urllib.request, urllib.parse, urllib.error
import json
from bs4 import BeautifulSoup
import requests
import re
import os
import pandas as pd
import numpy as np


In [324]:

movid=pd.read_csv("C:\\Users\\Jayant\\Desktop\\ml-latest-small\\links.csv",delimiter=',')
# movname=pd.read_csv("")
del movid["tmdbId"]
finalmovid=movid[0:1000]



In [366]:
data1=pd.read_csv("C:\\Users\\Jayant\\Desktop\\title_imdb.csv",delimiter=',')

In [368]:
data1.head()

,movieId,imdbId
0,1,114709
1,2,113497
2,3,113228
3,4,114885
4,5,113041


In [314]:

url="https://www.imdb.com/title/tt0"+str(id_each)
response = requests.get(url)
soup=BeautifulSoup(response.text,'html.parser')

In [315]:
lister_items = soup.find_all("div", class_=["title_wrapper","imdbRating","slate_wrapper","titleReviewBar","plot_summary_wrapper"])


In [316]:
#plot
#attribute eror
lister_items[3].find('div',class_="summary_text").text.strip()

'A hopeless romantic Chicago Transit Authority token collector is mistaken for the fiancée of a coma patient.'

In [317]:
lister_items[1].h1.text.replace(u'\xa0', u' ').strip()
#name

'While You Were Sleeping (1995)'

In [331]:
year=lister_items[1].find('span',id="titleYear").a.text
int(year)
#release year
#attribute error

1995

In [319]:
#imdb rating
float(lister_items[0].strong.text)


6.7

In [320]:
int(lister_items[4].find('span').text)
#metascore

#if exists then proceed
#valueerror attrubuteerror

67

In [321]:
#genres

covr=lister_items[1].find_all(href=re.compile("genres"))
genres=[covr[i].text for i in range(len(covr))]
genres

['Comedy', 'Drama', 'Romance']

In [322]:
votes=lister_items[0].find_all('span',itemprop="ratingCount")[0].text.strip()
int(votes.replace(",", ""))

82913

In [332]:
#poster
#typeerror
lister_items[2].a.img["src"].strip()


'https://m.media-amazon.com/images/M/MV5BMTU5NjU4NDUzMl5BMl5BanBnXkFtZTgwMjc4NDgzMTE@._V1_UX182_CR0,0,182,268_AL__QL50.jpg'

In [370]:
# Lists to store the scraped data in
imdb_id=[]
movid=[]

names = []
plot=[]
genres=[]

year = []
imdb_ratings = []
metascores = []
votes = []
urlimg=[]

  

In [369]:
data=data1.sample(frac=1)
data.head()

,movieId,imdbId
9487,170289,410650
7688,89492,1210166
5710,27822,374102
4117,5902,268126
7144,71462,1313104


In [354]:
from time import sleep
from random import randint

In [ ]:
count=300

for idm, row in data.iterrows():
    
    print(row['movieId'], row['imdbId'])
    if count<=0:
        break
        
    sleep(randint(1,3))
    
    url="https://www.imdb.com/title/tt0"+str(row['imdbId'])
    response = requests.get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    lister_items = soup.find_all("div", class_=["title_wrapper","imdbRating","slate_wrapper","titleReviewBar","plot_summary_wrapper"])
    
    try:
        movieid=row['movieId']
        imdbid= row['imdbId']
        
        name=lister_items[1].h1.text.replace(u'\xa0', u' ').strip()#name
        yr=int(lister_items[1].find('span',id="titleYear").a.text)#year
        imd_r=float(lister_items[0].strong.text)#imdbrating
        met_r=int(lister_items[4].find('span').text)#metascore
        
        covr=lister_items[1].find_all(href=re.compile("genres"))##
        
        genr=[covr[i].text for i in range(len(covr))]#genres
        temp_vt=lister_items[0].find_all('span',itemprop="ratingCount")[0].text.strip()##
        vots=int(temp_vt.replace(",", ""))#votes
        smallstory=lister_items[3].find('div',class_="summary_text").text.strip()#plot
        
        urlimgtemp=lister_items[2].a.img["src"].strip()#image url
    
    except:
        continue
          
        
    else:
        count=count-1
        print("okay",count,name)
        imdb_id.append(imdbid)
        movid.append(movieid)
        names.append(name)
        plot.append(smallstory)
        genres.append(genr)
        year.append(yr)
        imdb_ratings.append(imd_r)
        metascores.append(met_r)
        votes.append(vots)
        urlimg.append(urlimgtemp)

print("Done..")
        
    
        
       
        

    
   

In [393]:
finaldf = pd.DataFrame({'movid':movid,'imdb_id':imdb_id,'movie': names,
                       'year': year,
                       'imdb': imdb_ratings,
                       'metascore': metascores,
                       'votes': votes,'genres':genres,'plot':plot,'zurl':urlimg})
print(finaldf.info())
finaldf.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
genres       300 non-null object
imdb         300 non-null float64
imdb_id      300 non-null int64
metascore    300 non-null int64
movid        300 non-null int64
movie        300 non-null object
plot         300 non-null object
votes        300 non-null int64
year         300 non-null int64
zurl         300 non-null object
dtypes: float64(1), int64(5), object(4)
memory usage: 23.5+ KB
None


,genres,imdb,imdb_id,metascore,movid,movie,plot,votes,year,zurl
0,"[Biography, Drama, Sport]",7.6,1210166,87,89492,Moneyball (2011),Oakland A's general manager Billy Beane's succ...,320711,2011,https://m.media-amazon.com/images/M/MV5BMjAxOT...
1,"[Adventure, Drama, Horror]",5.7,374102,63,27822,Open Water (2003),Based on the true story of two scuba divers ac...,47529,2003,https://m.media-amazon.com/images/M/MV5BNjQ5OD...
2,"[Comedy, Drama]",7.7,268126,83,5902,Adaptation. (2002),A lovelorn screenwriter becomes desperate as h...,163178,2002,https://m.media-amazon.com/images/M/MV5BZjIwZW...
3,"[Documentary, Crime]",8.4,1313104,84,71462,The Cove (2009),"Using state-of-the-art equipment, a group of a...",44744,2009,https://m.media-amazon.com/images/M/MV5BMzM3NT...
4,"[Comedy, Romance]",6.4,2784678,81,118924,Top Five (2014),A comedian tries to make it as a serious actor...,24539,2014,https://m.media-amazon.com/images/M/MV5BMjIzNT...


In [6]:
#finaldf.to_csv("precog_imdb300.csv")
finaldf=pd.read_csv("C:\\Users\\Jayant\\Desktop\\precog\\precog_imdb300.csv",delimiter=',')

In [7]:
def save_poster(title,poster_url):
   
    poster_file_extension=poster_url.split('.')[-1]
   
    poster_data = urllib.request.urlopen(poster_url).read()
        
    savelocation=os.getcwd()+'\\'+'Posters'+'\\'
   
    if not os.path.isdir(savelocation):
        os.mkdir(savelocation)
    
    filename=savelocation+str(title)+'.'+poster_file_extension
    filename=filename.strip()
    jj=open(filename,'wb')
    jj.write(poster_data)
    jj.close()

In [8]:
for ini,ekk in finaldf.iterrows(): 
    title=ekk['movie']
    poster_url=ekk['zurl']
    try:
        save_poster(title,poster_url)
    except:
        print(title)
        continue
        
    

Are We There Yet? (2005)
Why Him? (2016)
